Time series, number of reviews per month

In [6]:
import os
import json
import re
import csv
import pandas as pd

In [7]:
APP_LIST = 'fertility'
PROJECT = 'fertility-16012022'
PATH_ANDROID = '../data/' + PROJECT + '/play-store/'
PATH_IOS = '../data/' + PROJECT + '/app-store/'
LIST = '../data/apps/' + APP_LIST + '.json'

In [8]:
with open(LIST) as json_file:
    APPS = json.load(json_file)

In [13]:
COLS = ["app", "lang", "year", "month", "size"]
ANDROID_REVIEWS = pd.DataFrame(columns=COLS)

for app in APPS['apps']:
    app_name_dashed = app["android"]["id"].replace(".", "-")
    for subdir, dirs, files in os.walk(PATH_ANDROID + 'reviews/' + app_name_dashed):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']

    #     print('Found directory: %s' % subdir)

        for fname in files:
    #         print('\t%s' % fname)
            reviews = pd.read_csv(subdir + '/' + fname)
        
            reviews['at']= pd.to_datetime(reviews['at'])
            reviews['year'] = reviews['at'].dt.year
            reviews['month'] = reviews['at'].dt.month
            
            count_series = reviews.groupby(['year', 'month']).size()
            count_series = count_series.to_frame(name = 'size').reset_index()
            count_series['app'] = app["android"]["id"]
            count_series['lang'] = fname.split(".")[0]

            ANDROID_REVIEWS = ANDROID_REVIEWS.append(count_series)

ANDROID_REVIEWS.to_csv(PATH_ANDROID + 'reviews-play-store-time-series.csv', index=False)

In [5]:
COLS = ["app", "country", "year", "month", "size"]
IOS_REVIEWS = pd.DataFrame(columns=COLS)

for app in APPS['apps']:
    for subdir, dirs, files in os.walk(PATH_IOS  + '/reviews/' + app['ios']['id']):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']

        for fname in files:
            reviews = pd.read_csv(subdir + '/' + fname)
            reviews['date']= pd.to_datetime(reviews['date'])
            reviews['year'] = reviews['date'].dt.year
            reviews['month'] = reviews['date'].dt.month
            
            count_series = reviews.groupby(['year', 'month']).size()
            count_series = count_series.to_frame(name = 'size').reset_index()
            count_series['app'] = os.path.basename(os.path.normpath(subdir))
            count_series['country'] = re.search('(.+?)(\.[^.]*$|$)', fname).group(1)
            
            IOS_REVIEWS = IOS_REVIEWS.append(count_series)
        

IOS_REVIEWS.to_csv(PATH_IOS + 'reviews-app-store-time-series.csv', index=False)